In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from lifelines import CoxPHFitter
from lifelines.statistics import proportional_hazard_test
from lifelines.utils import k_fold_cross_validation
from sklearn.model_selection import train_test_split


In [ ]:
# Obtener la ruta absoluta del directorio actual (donde está el script o notebook)
base_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Subimos un nivel

# Construir la ruta relativa a la carpeta "Datos"
ruta_bdd = os.path.join(base_dir, "Datos", "database.xlsx")

# Leer el archivo de datos
data = pd.read_excel(ruta_bdd, sheet_name=None)


In [ ]:
# Concatenamos los dataframes de cada key:
data = pd.concat(data.values(), ignore_index=True)
print(data.shape)
data.head()

In [ ]:
from lifelines import CoxPHFitter

# Seleccionar las columnas relevantes para el modelo
columns_to_use = ["dias_230", "Caudal", "TPH", "% Solido", "Presion", "uso_230", "TPH_acum", "solido_dias","solido_uso","carga_solidos_efectiva"]
df_model = data[columns_to_use + ["E"]].dropna()  # Eliminar filas con valores faltantes

# Ajustar el modelo de riesgos proporcionales de Cox
cph = CoxPHFitter()
cph.fit(df_model, duration_col="dias_230", event_col="E")

# Mostrar el resumen del modelo
cph.print_summary()


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Definir las variables predictoras
X = df_model.drop(columns=["dias_230", "E"])  

# Estandarizar las variables antes de aplicar PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Aplicar PCA (mantendremos suficientes componentes para capturar 95% de la varianza)
pca = PCA(n_components=0.95)  # Retiene el 95% de la varianza
X_pca = pca.fit_transform(X_scaled)

# Convertir el PCA a DataFrame
df_pca = pd.DataFrame(X_pca, columns=[f"PCA_{i+1}" for i in range(X_pca.shape[1])])

# Agregar las columnas de tiempo y E
df_pca["dias_230"] = df_model["dias_230"].values
df_pca["E"] = df_model["E"].values

# Ver la varianza explicada por cada componente
print("Varianza explicada por cada componente:", pca.explained_variance_ratio_)
print("Varianza total explicada:", pca.explained_variance_ratio_.sum())

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import numpy as np

# Definir modelo base (Regresión logística para clasificación)
modelo_base = LogisticRegression(max_iter=1000)

# Aplicar RFE para seleccionar las mejores características
rfe = RFE(estimator=modelo_base, n_features_to_select=5)  # Seleccionar 5 mejores variables
X_rfe = rfe.fit_transform(X_scaled, df_model["E"])  # RFE sobre datos estandarizados

# Obtener las variables seleccionadas
variables_seleccionadas = X.columns[rfe.support_]
print("Variables seleccionadas por RFE:", variables_seleccionadas)

# Crear un nuevo DataFrame con las variables seleccionadas
df_rfe = df_model[["dias_230", "E"] + list(variables_seleccionadas)]
